In [1]:
#导入要用到的模块
import numpy as np
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from sklearn import feature_extraction
import numpy as np  
import lda
import lda.datasets 
import os    
import sys  
import matplotlib  
import scipy  
import matplotlib.pyplot as plt  
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer 

In [2]:
review = open('../US.20180118.txt').read().split(','+'\"')

In [3]:
voc = open('../vocab_US.txt').read().split('\t')
voc = set(voc)
voc

{'   assembl ',
 '   handl',
 ' actual',
 ' app',
 ' beware',
 ' cool',
 ' easy ',
 ' funky ',
 ' impressed',
 ' money',
 ' packing',
 ' perfect',
 ' pictur',
 ' price',
 ' qualiti',
 ' quickly',
 ' recommend',
 ' recommend ',
 ' slow',
 ' soft',
 ' time',
 ' togeth',
 ' verified',
 'Amazon',
 'Barbie',
 'Disney',
 'Funko',
 'Hape',
 'KidKraft',
 'LEGO',
 'Marvel',
 'Spider-Man',
 'admit',
 'adorable',
 'arrive',
 'assembled',
 'awesome',
 'best',
 'big',
 'boring',
 'brand',
 'broke',
 'carrying',
 'certified',
 'challenging',
 'cheap',
 'colorful',
 'comfortable',
 'confusing',
 'craft',
 'creative',
 'cuddly ',
 'cute ',
 'description',
 'design',
 'detail',
 'disappointed',
 'disgusting',
 'emoji ',
 'engaging',
 'exactly ',
 'excellent',
 'excited',
 'expect',
 'expected',
 'expensive',
 'fake',
 'fantastic',
 'fascinated',
 'fast',
 'favorite',
 'fossil ',
 'free',
 'fun',
 'glad',
 'glad ',
 'gorgeous ',
 'great',
 'happi',
 'happy',
 'hilarious',
 'hit',
 'hold',
 'instructions

In [4]:
# 载入 nltk 的英文停用词作为“stopwords”变量
stopwords = nltk.corpus.stopwords.words('english')

In [5]:
# 载入 nltk 的 SnowballStemmer 作为“stemmer”变量(把一些動詞形式轉換成原型)
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [6]:
#分词
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

In [7]:
# 分詞器
def tokenize_and_stem(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    #过滤掉非字母，比如数字和间隔等
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [8]:
#文本准备
from gensim import corpora, models, similarities

#remove proper names
preprocess = [strip_proppers(doc) for doc in review]

In [9]:
tokenized_text = [tokenize_and_stem(text) for text in preprocess]

In [10]:
#text
# texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

In [11]:
#voc
texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

In [12]:
#词典和语料库准备
dictionary = corpora.Dictionary(texts) #构造词典
# dictionary.filter_extremes(no_below=1, no_above=0.8)#去高频词

corpus = [dictionary.doc2bow(text) for text in texts] #构造语料库

In [13]:
len(corpus)

3728

In [14]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 2),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1)]

In [15]:
#训练一个LDA模型
%time lda = models.LdaModel(corpus, num_topics=10, id2word=dictionary, update_every=5,passes=50)

CPU times: user 11min 45s, sys: 7.19 s, total: 11min 52s
Wall time: 12min 57s


In [16]:
print(lda[corpus[0]])

[(3, 0.58218771), (8, 0.3870368)]


In [17]:
topics = lda.print_topics(10)#结果中的主题

In [18]:
lda.print_topic(3, topn=100)

'0.027*"love" + 0.022*"\'s" + 0.015*"one" + 0.014*"n\'t" + 0.013*"cute" + 0.013*"gift" + 0.011*"littl" + 0.011*"great" + 0.010*"daughter" + 0.010*"veri" + 0.010*"like" + 0.008*"got" + 0.008*"product" + 0.008*"look" + 0.008*"price" + 0.007*"get" + 0.007*"perfect" + 0.007*"good" + 0.007*"qualiti" + 0.006*"kid" + 0.006*"bought" + 0.006*"birthday" + 0.006*"old" + 0.006*"would" + 0.006*"soft" + 0.006*"made" + 0.006*"realli" + 0.006*"expect" + 0.005*"year" + 0.005*"purchas" + 0.005*"happi" + 0.005*"came" + 0.005*"toy" + 0.005*"son" + 0.005*"play" + 0.005*"buy" + 0.005*"box" + 0.005*"packag" + 0.005*"pictur" + 0.005*"super" + 0.005*"well" + 0.005*"order" + 0.004*"open" + 0.004*"much" + 0.004*"work" + 0.004*"doe" + 0.004*"receiv" + 0.004*"know" + 0.004*"want" + 0.004*"still" + 0.004*"size" + 0.004*"becaus" + 0.004*"day" + 0.004*"figur" + 0.004*"pretti" + 0.004*"collect" + 0.004*"even" + 0.004*"make" + 0.004*"item" + 0.003*"arriv" + 0.003*"sure" + 0.003*"parti" + 0.003*"girl" + 0.003*"ador" + 0

In [19]:
topics #主题

[(0,
  '0.014*"n\'t" + 0.013*"\'s" + 0.012*"one" + 0.012*"use" + 0.011*"get" + 0.010*"year" + 0.009*"veri" + 0.009*"like" + 0.008*"wheel" + 0.008*"ride"'),
 (1,
  '0.039*"color" + 0.018*"use" + 0.012*"marker" + 0.011*"n\'t" + 0.010*"like" + 0.009*"paper" + 0.009*"littl" + 0.009*"great" + 0.008*"water" + 0.008*"set"'),
 (2,
  '0.022*"\'s" + 0.020*"n\'t" + 0.017*"toy" + 0.013*"babi" + 0.011*"old" + 0.010*"well" + 0.010*"water" + 0.010*"bath" + 0.009*"work" + 0.008*"month"'),
 (3,
  '0.027*"love" + 0.022*"\'s" + 0.015*"one" + 0.014*"n\'t" + 0.013*"cute" + 0.013*"gift" + 0.011*"littl" + 0.011*"great" + 0.010*"daughter" + 0.010*"veri"'),
 (4,
  '0.018*"gun" + 0.014*"dart" + 0.011*"veri" + 0.009*"use" + 0.009*"n\'t" + 0.009*"wagon" + 0.008*"nerf" + 0.008*"shoot" + 0.008*"one" + 0.008*"blaster"'),
 (5,
  '0.031*"love" + 0.021*"toy" + 0.020*"old" + 0.019*"play" + 0.014*"year" + 0.011*"\'s" + 0.010*"great" + 0.010*"littl" + 0.010*"one" + 0.009*"like"'),
 (6,
  '0.029*"doll" + 0.013*"bubbl" + 0.

In [20]:
topics_matrix = lda.show_topics(formatted=False,num_words=20)

In [21]:
topics_matrix

[(0,
  [("n't", 0.014482182),
   ("'s", 0.012800287),
   ('one', 0.011909137),
   ('use', 0.011806506),
   ('get', 0.011295191),
   ('year', 0.01045979),
   ('veri', 0.0091683436),
   ('like', 0.0086478097),
   ('wheel', 0.0078730639),
   ('ride', 0.0078698229),
   ('old', 0.0076374584),
   ('love', 0.0071162214),
   ('kid', 0.0068469276),
   ('great', 0.0067579173),
   ('would', 0.0064065093),
   ('scooter', 0.0063146441),
   ('bike', 0.0061832438),
   ('daughter', 0.0060586617),
   ('littl', 0.0057878182),
   ('fit', 0.0056720874)]),
 (1,
  [('color', 0.038936321),
   ('use', 0.018015593),
   ('marker', 0.012246297),
   ("n't", 0.010520052),
   ('like', 0.0097324755),
   ('paper', 0.0094160587),
   ('littl', 0.0093773995),
   ('great', 0.0089698536),
   ('water', 0.0082243979),
   ('set', 0.0081822099),
   ('veri', 0.0080565251),
   ("'s", 0.0075350506),
   ('work', 0.0072164559),
   ('pencil', 0.0069463584),
   ('kid', 0.0069025788),
   ('project', 0.0067478972),
   ('dri', 0.006718